In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

/opt/miniconda3/envs/test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('Playlist_data_with_lyrics.csv')
artists = pd.read_csv('artists.csv')

In [3]:
df

,Playlist_ID,Playlist_Name,Playlist_Songs,Playlist_Artists,Playlist_Song_IDs
0,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"['Into You', 'Glad You Came', 'Dark Horse', 'W...","['Ariana Grande', 'The Wanted', 'Katy Perry', ...","['2meEiZKWkiN28gITzFwQo5', '1OXfWI3FQMdsKKC6lk..."
1,34NbomaTu7YuOYnky8nLXL,Pop Hits 2025 (Top 50),"['Die With A Smile', 'APT.', 'Espresso', ""we c...","['Lady Gaga', 'ROSÉ', 'Sabrina Carpenter', 'Ar...","['2plbrEY59IikOBgBGLjaoe', '5vNRhkKd0yEAg8suGB..."
2,4Jb4PDWREzNnbZcOHPcZPy,COUNTRY HITS 2025 🔥 New Country Songs + Top Hits,"['I Had Some Help (Feat. Morgan Wallen)', ""Aus...","['Post Malone', 'Dasha', 'mgk', 'Dylan Marlowe...","['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXT..."
3,1Cgey68pUlQGsCPI2wJuxr,Best of 2025 🔥 Most Popular Hits 2025 Hits,"['APT.', 'Anxiety', 'Die With A Smile', 'Messy...","['ROSÉ', 'Doechii', 'Lady Gaga', 'Lola Young',...","['5vNRhkKd0yEAg8suGBpjeY', '1musbempyJAw5gfSKZ..."
4,2L2HwKRvUgBv1YetudaRI3,Pop 2000-2010 Bangers,"['Whatcha Say', 'Airplanes (feat. Hayley Willi...","['Jason Derulo', 'B.o.B', 'Bruno Mars', 'Tinch...","['7xkQdy0cy5ymoWT7nedvLz', '1QnvpPFP4Q3FHbDchq..."
...,...,...,...,...,...
1821,6qyyWabXbr3InAGVlY6xb5,Hype Russian songs,"['ДУЛО', 'Новая волна', 'Cristal & МОЁТ', 'FEE...","['MORGENSHTERN', 'DJ SMASH', 'MORGENSHTERN', '...","['2Db7e6H4R5XXnyuTtxUggp', '7rRzijIIKQ8GntJ8IU..."
1822,1n7fMfBvijF4JBkFekFVpE,Learn Russian in Your Car,"['Introduction', 'Lesson 1 - Basics', 'Lesson ...","['Henry Raymond Jr.', 'Henry Raymond Jr.', 'He...","['4bmotjQ98b8OfUvHvSKHTh', '263DVJyCMhSgLy7UNj..."
1823,2Y8I4KdIOYCLZoEmrUQj1o,Russian Roulette,"['House Of The Rising Sun', 'KICK BACK', 'RUN ...","['The Animals', 'Kenshi Yonezu', 'iamjakehill'...","['7BY005dacJkbO6EPiOh2wb', '3khEEPRyBeOUabbmOP..."
1824,5y8oqVJhiYEhAm3xkv7oob,russian roulette,"['Die With A Smile', 'Die With A Smile', 'Die ...","['Lady Gaga', 'Lady Gaga', 'Lady Gaga', 'KSI',...","['2plbrEY59IikOBgBGLjaoe', '2plbrEY59IikOBgBGL..."


In [3]:
# Convert stringified lists to Python lists if needed
import ast
for col in ['Playlist_Songs', 'Playlist_Artists', 'Playlist_Song_IDs']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Build target output string: "Song1 by Artist1, Song2 by Artist2, ..."
def build_output_text(row):
    songs = row['Playlist_Songs']
    artists = row['Playlist_Artists']
    paired = [f"{s} by {a}" for s, a in zip(songs, artists)]
    return ', '.join(paired)

# Build final dataset
df['input_text'] = df['Playlist_Name']
df['target_text'] = df.apply(build_output_text, axis=1)

# Split into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['input_text'].tolist(),
    df['target_text'].tolist(),
    test_size=0.2
)

train_data = Dataset.from_dict({'input_text': train_texts, 'label': train_labels})
val_data = Dataset.from_dict({'input_text': val_texts, 'label': val_labels})

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenize input and label
def tokenize(batch):
    input_encodings = tokenizer(batch['input_text'], padding="max_length", truncation=True, max_length=64)
    label_encodings = tokenizer(batch['label'], padding="max_length", truncation=True, max_length=256)
    input_encodings["labels"] = label_encodings["input_ids"]
    return input_encodings

train_data = train_data.map(tokenize, batched=True)
val_data = val_data.map(tokenize, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|███████████████████████████| 366/366 [00:00<00:00, 1082.65 examples/s]


In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./t5_playlist_model",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
def generate_playlist(prompt, max_length=128):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length=max_length, temperature=0.8)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded

# Example prompt
prompt = "Road trip anthems for 2020s pop hits"
print(generate_playlist(prompt))

# GPT Model

In [6]:
import pandas as pd
from datasets import Dataset
import ast

# Load and parse the dataset

# Convert stringified lists to actual lists
for col in ['Playlist_Songs', 'Playlist_Artists']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Create prompt + output text for GPT-2 training
def format_example(row):
    song_lines = [f"{s} - {a}" for s, a in zip(row['Playlist_Songs'], row['Playlist_Artists'])]
    return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(song_lines)

df['text'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": df['text'].tolist()})

In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # In case we add special tokens

# Tokenize
def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # 🔥 Add this line
    return encodings
    
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████████████████████| 1826/1826 [00:03<00:00, 540.88 examples/s]


In [13]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./gpt2_playlist_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True if torch.cuda.is_available() else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.733000
100,3.368300
150,3.211100
200,3.343100
250,3.254700
300,3.105400
350,3.167800
400,3.231800
450,3.126600
500,3.014500


TrainOutput(global_step=2739, training_loss=2.842794233797762, metrics={'train_runtime': 744.0231, 'train_samples_per_second': 7.363, 'train_steps_per_second': 3.681, 'total_flos': 1431357751296000.0, 'train_loss': 2.842794233797762, 'epoch': 3.0})

In [19]:
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('mps'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "90s Grunge"
print(generate_playlist(prompt))

American Gods - Neil Diamond
Born to Run - Daryl Hall & John Mellencamp
The Greatest Show - Daryl Hall & John Mellencamp
Mr. Blue Sky - Daryl Hall & John Mellencamp
Cars - Daryl Hall & John Mellencamp
The Good, The Bad And The Ugly - Daryl Hall & John Mellencamp
All of the Above - Daryl Hall & John Mellencamp
Locked in the Cold - Daryl Hall & John Mellencamp
So Here's To You - Daryl Hall & John Mellencamp
The Last Night - Daryl Hall & John Mellencamp
The Power Of Love - Daryl Hall & John Mellencamp
You're a Fine Girl - Daryl Hall & John Mellencamp
The Night We Met - Daryl Hall & John Mellencamp
The Night We Met Tonight - Daryl Hall & John Mellencamp
We Are The World - Bruce Spring


1826